## Prueba de modelos ##

### Version 4: Modelo con 2 DER, planificación anual, granularidad horaria ###

Si el modelo tiene que optimizar el balance por horas, tiene 8640 VD, pero sigue teniendo 2 VD binarias

In [73]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
%autoreload 2

In [75]:
import pandapower as pp

In [76]:
import os

In [77]:
import pandas as pd

In [78]:
import pyomo.environ as pe

In [79]:
import math

In [80]:
import random

In [81]:
import itertools

In [82]:
import matplotlib.pyplot as plt

In [83]:
import numpy as np

In [84]:
import ModelWriters.SimpleBusbar

Variables que define el modelo:

y para el año

d para el día del año

h para la hora del día

dt la granularidad del modelo, en horas


temp la temperatura

wv la velocidad del viento

I la irradiación solar


eg el crecimiento económico en pu respecto al año 0


Estas variables llegan como parámetros en forma de diccionario en el argumento 'model_status' (ver este nombre)

Todas las funciones reciben este argumento, la lógica de la función indica que valor retorna. Pr ejemplo, si la granularidad es 24 h, debe retornar el valor medio del parámetro producido.

## Construcción de Escenarios

In [85]:
import ModelWriters.Simulation.Solar as Solar
import ModelWriters.Simulation.Demand as Demand

In [86]:
dias = 1
años = 5

In [87]:
#el rango de horas:
T_i = range(24)

#el rango de dias:
Dias_i = range(dias)

Escenarios_i = range(len(T_i)*len(Dias_i))

In [88]:
m_s_base = [{'y': 0, 'd': 0, 'dd':1.0, 'h': 0.0, 'dt': 1.0, 'temp': 20.0, 'I':1000.0, 'wv': 10.0, 'eg': 1.0}]

registros = len(Dias_i)*len(T_i)

Escenarios =  pd.DataFrame(m_s_base, index = Escenarios_i)

reg = 0
for d in Dias_i:
    for h in T_i:
        Escenarios['d'][reg] = d
        Escenarios['h'][reg] = h
        Escenarios['I'][reg] = Solar.solar_irradiance_seasoned_randomized(d, h)
        Escenarios['wv'][reg] = random.uniform(1.0, 20.0)
        reg += 1

In [89]:
Escenarios

,y,d,dd,h,dt,temp,I,wv,eg
0,0,0,1.0,0.0,1.0,20.0,0.000000,17.296744,1.0
1,0,0,1.0,1.0,1.0,20.0,0.000000,16.165020,1.0
2,0,0,1.0,2.0,1.0,20.0,0.000000,7.121180,1.0
3,0,0,1.0,3.0,1.0,20.0,0.000000,9.641425,1.0
4,0,0,1.0,4.0,1.0,20.0,0.000000,2.304743,1.0
5,0,0,1.0,5.0,1.0,20.0,0.000000,12.537768,1.0
6,0,0,1.0,6.0,1.0,20.0,3.036046,16.622207,1.0
7,0,0,1.0,7.0,1.0,20.0,45.782496,9.500704,1.0
8,0,0,1.0,8.0,1.0,20.0,73.636263,18.340602,1.0
9,0,0,1.0,9.0,1.0,20.0,125.513279,4.117449,1.0


## Model Tests

In [91]:
from ModelWriters.Resources.ExtGrid import ExtGrid

In [92]:
from ModelWriters.Resources.Load import Load

In [93]:
from ModelWriters.Resources.BaseGen import Generator, DiscreteGenerator

In [94]:
from ModelWriters.Resources.Storage import Storage

In [100]:
model = pe.ConcreteModel()

In [21]:
exg = ExtGrid('EXT')

In [22]:
bg = Generator('GEN')

In [96]:
dg = DiscreteGenerator('DGEN')

In [23]:
load = Load('LOAD')

In [68]:
storage = Storage('STORAGE')

In [25]:
exg['pr_mw'] = 5.0
exg['ic_0_mu'] = 1.2
exg['ic_1_mu'] = 0.2
exg['oc_0_mu'] = 0.001
exg['oc_1_mu'] = 0.005

#exg.pa_pu = wind_output

In [37]:
load['pr_mw'] = .5
load['ic_0_mu'] = 1.2
load['ic_1_mu'] = 0.2
load['oc_0_mu'] = 0.001
load['oc_1_mu'] = 0.005
load['pa_pu'] = Demand.scalonated_seasoned_randomized_demand

#exg.pa_pu = wind_output

In [97]:
dg['ic_0_mu'] = 0.44
dg['ic_1_mu'] = 0.2
dg['unit_cost_mu'] = 0.3
dg['oc_0_mu'] = 0.001
dg['oc_1_mu'] = 0.005

In [38]:
exg.initialize_model(model, Escenarios)

In [39]:
bg['pr_mw'] = 5.0
bg['ic_0_mu'] = 1.2
bg['ic_1_mu'] = 0.2
bg['oc_0_mu'] = 0.001
bg['oc_1_mu'] = 0.005


In [40]:
bg.initialize_model(model, Escenarios)

In [41]:
load.initialize_model(model, Escenarios)

In [101]:
dg.initialize_model(model, Escenarios)

In [42]:
load.available_power(10)

0.2623666302630328

In [43]:
bg.available_power(15)

In [44]:
print(bg.initial_cost())

1.2*GEN_create + 0.2*GEN_pr_mw


In [45]:
print(load.initial_cost())

1.3


In [46]:
print(bg.operating_cost(10))

0.001 + 0.005*GEN_p_mw[10]


In [47]:
print(load.operating_cost(10))

0.001 + 0.005*LOAD_p_mw[10]


In [48]:
print(bg.active_power(10))

GEN_p_mw[10]


In [49]:
print(load.active_power(10))

-0.2633838307853684


In [69]:
storage.initialize_model(model, Escenarios)

In [102]:
model.pprint()

2 Set Declarations
    DGEN_p_constraint_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   24 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23}
    DGEN_p_mw_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   24 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23}

4 Var Declarations
    DGEN_create : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :     1 : False :  True : Binary
    DGEN_p_mw : Size=24, Index=DGEN_p_mw_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :